In [1]:
from utils.api_reader import SigmaWalletReader

ImportError: Unable to import required dependencies:
numpy: No module named 'numpy'

In [ ]:
reader = SigmaWalletReader(config_path="../conf")

In [ ]:
data = reader.get_api_data('http://15.204.211.130:4000/api/pools/ErgoSigmanauts/miners/')
data

In [ ]:
miner_ls = []
for sample in data:
    miner_ls.append(sample['miner'])


miner_ls

In [ ]:
miners = {}
for sample in data:
    miners[sample['miner']] = 0
for key in miners.keys():
    url = '{}/{}'.format('http://15.204.211.130:4000/api/pools/ErgoSigmanauts/miners', key)
    sample = reader.get_api_data(url)
    miners[key] = sample['pendingShares']
miners

In [ ]:
# Total of all values
total = sum(miners.values())

# Assume a reward variable, say 10000 for this example
reward = 30

# Calculate the percentage for each key based on its value and then determine the reward based on this percentage
rewards = {key: (value / total) * reward for key, value in miners.items()}

rewards

In [ ]:
import pandas as pd
rewards_df = pd.DataFrame(list(rewards.items()), columns=['miner', 'reward'])
rewards_df

In [ ]:
url = 'http://15.204.211.130:4000/api/pools/ErgoSigmanauts/miners/9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk'
data = reader.get_api_data(url)
samples = data['performanceSamples']
samples

In [ ]:
df = reader.get_miner_samples(wallet='9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk')

In [ ]:
df.empty

In [ ]:
d = {'a': [1], 
     'b': [2]}
pd.DataFrame.from_dict(d)

In [ ]:
import plotly.express as px

fig = px.line(df, x='created', y='hashrate', color='worker', title='Hashrate Over Time for Each Worker')
fig.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import random

def generate_random_variable_list(n):
    random_variable_list = []
    for i in range(n):
        random_variable = random.uniform(0.001, 5)
        random_variable_list.append(random_variable)
    return random_variable_list

n = 10 # number of random variables
y = generate_random_variable_list(n)

x = [*range(0, n)]

df = pd.DataFrame({'x': x, 'y':y})
df['z'] = df['y'].expanding().mean()


plt.plot(df['x'], df['y'], label='y')
plt.plot(df['x'], df['z'], label='z')
plt.xlabel('x')
plt.ylabel('y and z')
plt.title('Plotting y and z over x')
plt.legend()
plt.show()

In [ ]:
df

In [ ]:
df

In [ ]:
n = df.melt(id_vars=['x'])
n

In [ ]:
random.randrange(0,5)

In [ ]:
url ='http://15.204.211.130:4000/api/pools/ErgoSigmanauts'

pool = reader.get_api_data(url)['pool']
pool.keys()

In [ ]:
pool['poolEffort']

In [ ]:
payment_data = pool['paymentProcessing']
del payment_data['enabled']
del payment_data['payoutSchemeConfig']
del payment_data['extra']

In [ ]:
payment_data

In [ ]:
port_data = pool['ports']

ls = []
for port in port_data:
    temp = port_data[port]
    if 'pikes_peak' in temp['name']:
        high_or_low = 'Greater Than 10GH/s'
    else:
        high_or_low = 'Lower Than 10GH/s'
    ls.append([temp['name'], port, high_or_low, temp['tls']])
df = pd.DataFrame(ls, columns=['Name', 'Port', 'Hashrate Threshold', 'TLS'])
df

In [ ]:
pool_fee = pool['poolFeePercent']
pool_stats = pool['poolStats']
net_stats = pool['networkStats']
total_paid = pool['totalPaid']
total_blocks = pool['totalBlocks']
last_block_found = pool['lastPoolBlockTime']
pool_effort = pool['poolEffort']

In [ ]:
net_stats
del net_stats['connectedPeers']
del net_stats['rewardType']
del net_stats['networkType']

In [ ]:
pool_stats

In [ ]:
import dash

In [ ]:
data = reader.get_front_page_data()

In [ ]:
data['poolHashrate'] = data['poolHashrate'] / 1e9 #GIGA
data['poolHashrate'] 

In [ ]:
data

In [ ]:
last_block_found

In [ ]:
df = reader.get_all_miner_data('9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk')
df

In [ ]:
df[df.wallet == '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk']

In [ ]:
from datetime import datetime
import pytz

def calculate_mining_effort(network_difficulty, network_hashrate, pool_hashrate, last_block_timestamp):
    """
    Calculate the mining effort for the pool to find a block on Ergo blockchain based on the given timestamp.
    
    :param network_difficulty: The current difficulty of the Ergo network.
    :param network_hashrate: The total hash rate of the Ergo network (in hashes per second).
    :param pool_hashrate: The hash rate of the mining pool (in hashes per second).
    :param last_block_timestamp: Timestamp of the last block found in ISO 8601 format.
    :return: The estimated mining effort for the pool.
    """
    # Parse the last block timestamp
    time_format = '%Y-%m-%d %H:%M:%S' 
    last_block_time = datetime.strptime(last_block_timestamp, time_format)
    last_block_time = last_block_time.replace(tzinfo=pytz.utc)  # Assume the timestamp is in UTC
    
    # Get the current time in UTC
    now = datetime.now(pytz.utc)
    
    # Calculate the time difference in seconds
    time_since_last_block = (now - last_block_time).total_seconds()
    
    # Hashes to find a block at current difficulty
    hashes_to_find_block = network_difficulty  # This is a simplification
    
    # Total hashes by the network in the time since last block
    total_network_hashes = network_hashrate * time_since_last_block
    
    # Pool's share of the total network hashes
    pool_share_of_hashes = (pool_hashrate / network_hashrate) * total_network_hashes
    
    # Effort is the pool's share of hashes divided by the number of hashes to find a block
    effort = pool_share_of_hashes / hashes_to_find_block
    
    return effort

# Example usage:
network_difficulty = 1.83e15  # Example difficulty
network_hashrate = 15.4e12  # Example total network hash rate (1 PH/s)
pool_hashrate = 13.3e9  # Example pool hash rate (100 TH/s)
last_block_timestamp = "2024-03-31 07:40:29"

effort = calculate_mining_effort(network_difficulty, network_hashrate, pool_hashrate, last_block_timestamp)
print(f"The estimated mining effort for the pool since the last block is: {effort}")

In [ ]:
from datetime import datetime

date_time_str = '2024-03-19 04:19:20'
format = '%Y-%m-%d %H:%M:%S'  # Added a space between '%d' and '%H'
date_time_obj = datetime.strptime(date_time_str, format)

print(date_time_obj)

In [ ]:
type(date_time_str)

In [ ]:
miners = reader.get_miner_ls()
data = []
for miner in miners:
    temp = reader.get_miner_samples(miner)
    data.append(temp)

df = pd.concat(data)

In [ ]:
df

In [ ]:
ls = []
for date in df.created.unique():
    temp = df[df.created == date]
    ls.append([date, temp.hashrate.sum() /1e9])

n = pd.DataFrame(ls, columns=['Date', 'Hashrate'])

In [ ]:
n

In [ ]:
df = reader.get_all_miner_data('9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk')
ls = []
for miner in df.miner.unique():
    temp = df[df.miner == miner]
    ls.append([miner, temp.hashrate.sum(), temp.sharesPerSecond.sum()])

d = pd.DataFrame(ls, columns=['Miner', 'Hashrate', 'SharesPerSecond'])
d.Miner.unique()[0]

In [ ]:
latest = max(df.created)
latest_data = df[df.created == latest]
my_data = latest_data[latest_data.my_wallet == True]
my_data = my_data.filter(['worker', 'hashrate', 'sharesPerSecond'])
total_hash = my_data.hashrate.sum()
total_shares = my_data.sharesPerSecond.sum()
ls = ['Totals', total_hash, total_shares]

d = pd.DataFrame([ls], columns=['worker', 'hashrate', 'sharesPerSecond'])
data = pd.concat([my_data, d])
data

In [ ]:
data.hashrate

In [ ]:
data['ttf'] = [reader.calculate_time_to_find_block(network_difficulty, network_hashrate, hash, latest) for hash in data.hashrate]
data['effort'] = [reader.calculate_mining_effort(network_difficulty, network_hashrate, hash, latest) for hash in data.hashrate]

In [ ]:
wallet ='9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk'

In [ ]:
reader.get_all_miner_data(wallet)

In [ ]:
pool_df, _ = reader.get_pool_stats(wallet) 
mining_df, performance_df = reader.get_mining_stats(wallet)

In [ ]:
mining_df

In [ ]:
miners = reader.get_miner_ls()
ls = []
for miner in miners:
    df, _ = reader.get_mining_stats(miner)
    shares = df[df['Mining Stats'] == 'pendingShares'].Values[0]
    ls.append([miner, shares])

n_df = pd.DataFrame(ls, columns=['Miner', 'Shares'])


In [ ]:
total = n_df.Shares.sum()
n_df['p'] = [shares / total for shares in n_df.Shares]
n_df['r'] = n_df['p'] * 30

In [ ]:
n_df
my_df = n_df[n_df.Miner == wallet]
participation = my_df.p


In [ ]:
participation.values[0]